### Quantile and Bucket Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
frame = pd.DataFrame({"data1":np.random.standard_normal(1000),
                     "data2":np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,0.026093,1.002658
1,0.350160,-1.342018
2,1.162536,-0.107566
3,-0.948141,0.132286
4,-0.135787,1.108011


In [3]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head()

0    (-1.44, 0.357]
1    (-1.44, 0.357]
2    (0.357, 2.155]
3    (-1.44, 0.357]
4    (-1.44, 0.357]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.245, -1.44] < (-1.44, 0.357] < (0.357, 2.155] < (2.155, 3.952]]

In [4]:
def get_stats(group):
    return pd.DataFrame({"min":group.min(), 
                        "max": group.max(), 
                        "count":group.count(), 
                        "mean":group.mean()})

In [5]:
grouped = frame.groupby(quartiles)

In [6]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                     
(-3.245, -1.44] data1 -3.237896 -1.445520     84 -1.977893
                data2 -2.111320  2.026536     84 -0.047668
(-1.44, 0.357]  data1 -1.436996  0.353196    559 -0.386007
                data2 -2.751680  3.114451    559 -0.015593
(0.357, 2.155]  data1  0.358005  2.130342    335  1.010737
                data2 -3.113069  3.091912    335 -0.017285
(2.155, 3.952]  data1  2.216241  3.952293     22  2.686205
                data2 -2.788988  1.349951     22 -0.231973

In [7]:
grouped.agg(["min", "max", "count", "mean", "std"])

data1                                         data2  \
                      min       max count      mean       std       min   
data1                                                                     
(-3.245, -1.44] -3.237896 -1.445520    84 -1.977893  0.478215 -2.111320   
(-1.44, 0.357]  -1.436996  0.353196   559 -0.386007  0.483828 -2.751680   
(0.357, 2.155]   0.358005  2.130342   335  1.010737  0.452095 -3.113069   
(2.155, 3.952]   2.216241  3.952293    22  2.686205  0.530225 -2.788988   

                                                     
                      max count      mean       std  
data1                                                
(-3.245, -1.44]  2.026536    84 -0.047668  0.856669  
(-1.44, 0.357]   3.114451   559 -0.015593  1.042148  
(0.357, 2.155]   3.091912   335 -0.017285  0.986780  
(2.155, 3.952]   1.349951    22 -0.231973  0.995571

### Filling missing value with group specific value

In [8]:
s = pd.Series(np.random.standard_normal(6))
s

0    1.612343
1    0.132441
2   -0.139647
3    1.399873
4   -1.050054
5    1.461201
dtype: float64

In [9]:
s[::2] = np.nan
s

0         NaN
1    0.132441
2         NaN
3    1.399873
4         NaN
5    1.461201
dtype: float64

In [10]:
s.mean()

0.9978383225427164

In [11]:
s.fillna(s.mean())

0    0.997838
1    0.132441
2    0.997838
3    1.399873
4    0.997838
5    1.461201
dtype: float64

In [12]:
states = ["Ohio", "New York", "Vermont", "Florida","Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East","West", "West", "West", "West"]

In [13]:
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio          0.072074
New York     -0.345022
Vermont       1.571826
Florida      -0.161381
Oregon       -0.699437
Nevada        1.537126
California   -0.082399
Idaho         1.045868
dtype: float64

In [14]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          0.072074
New York     -0.345022
Vermont            NaN
Florida      -0.161381
Oregon       -0.699437
Nevada             NaN
California   -0.082399
Idaho              NaN
dtype: float64

In [15]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [16]:
grouped = data.groupby(group_key)

In [18]:
grouped.agg(["size", "count", "mean", "min"])

,size,count,mean,min
East,4,3,-0.144776,-0.345022
West,4,2,-0.390918,-0.699437


In [19]:
def fill_mean(group):
    return group.fillna(group.mean())

In [20]:
grouped.apply(fill_mean)

East  Ohio          0.072074
      New York     -0.345022
      Vermont      -0.144776
      Florida      -0.161381
West  Oregon       -0.699437
      Nevada       -0.390918
      California   -0.082399
      Idaho        -0.390918
dtype: float64

### Example - Group Weighted average and correlation

In [21]:
df = pd.DataFrame({"category":["a", "a", "a", "a","b", "b", "b", "b"], 
                  "data":np.random.standard_normal(8),
                  "weight":np.random.uniform(8)})
df

,category,data,weight
0,a,-1.137222,1.953958
1,a,-1.453884,1.953958
2,a,0.290464,1.953958
3,a,1.056875,1.953958
4,b,-0.107218,1.953958
5,b,0.984276,1.953958
6,b,-1.237963,1.953958
7,b,0.044456,1.953958


In [22]:
grouped = df.groupby("category")

In [23]:
def get_wavg(group):
    return np.average(group["data"], weights=group["weight"])

In [25]:
grouped.apply(get_wavg) #Weighted average by category

category
a   -0.310942
b   -0.079112
dtype: float64

In [26]:
close_px = pd.read_csv("stock_px.csv", parse_dates=True, index_col=0)

In [27]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [28]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [29]:
rets = close_px.pct_change().dropna()

In [30]:
def get_year(x):
    return x.year

In [31]:
by_year = rets.groupby(get_year)

In [32]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


## Group Transformation and "Unwrapped" GroupBys

In [33]:
df =pd.DataFrame({"key":["a","b","c"]*4, 
                 "value":np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [34]:
g = df.groupby("key")["value"]

In [35]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [37]:
def get_mean(group):
    return group.mean()

In [38]:
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [39]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [40]:
def times_two(x):
    return x*2

In [41]:
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [42]:
def get_rank(group):
    return group.rank(ascending = False)

In [43]:
g.transform(get_rank)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [44]:
def normalize(group):
    return (group - group.mean())/group.std()

In [45]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [46]:
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [47]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [48]:
#Here, we are doing arithmetic between the outputs of multiple GroupBy operations
#instead of writing a function and passing it to groupby(...).apply. That is what is
#meant by “unwrapped.”

normalize = (df['value'] - g.transform('mean'))/g.transform("std")

In [49]:
normalize

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### Pivot tables and crosstab

In [50]:
tips = pd.read_csv("tips.csv")
tips = tips.drop(['sex','time'], axis=1)
tips["tip_pct"] = tips["tip"]/tips["total_bill"]

In [51]:
tips.head()

,total_bill,tip,smoker,day,size,tip_pct
0,16.99,1.01,No,Sun,2,0.059447
1,10.34,1.66,No,Sun,3,0.160542
2,21.01,3.50,No,Sun,3,0.166587
3,23.68,3.31,No,Sun,2,0.139780
4,24.59,3.61,No,Sun,4,0.146808


In [52]:
tips.pivot_table(index=["day","smoker"]) #mean default

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [53]:
tips.pivot_table(index=["day"], columns="smoker", values=["size"])

size          
smoker        No       Yes
day                       
Fri     2.250000  2.066667
Sat     2.555556  2.476190
Sun     2.929825  2.578947
Thur    2.488889  2.352941

In [54]:
tips.pivot_table(index=["day"], columns="smoker", values=["size"], margins=True)
#All row and column labels

size                    
smoker        No       Yes       All
day                                 
Fri     2.250000  2.066667  2.105263
Sat     2.555556  2.476190  2.517241
Sun     2.929825  2.578947  2.842105
Thur    2.488889  2.352941  2.451613
All     2.668874  2.408602  2.569672

In [56]:
tips.pivot_table(index=["day"], columns="smoker", values=["size"], margins=True, aggfunc=len)

size         
smoker   No Yes  All
day                 
Fri       4  15   19
Sat      45  42   87
Sun      57  19   76
Thur     45  17   62
All     151  93  244

In [58]:
tips.pivot_table(index=["size", "smoker"], columns="day", values=["tip_pct"], margins=True, fill_value=10)

tip_pct                                           
day                Fri        Sat        Sun       Thur       All
size smoker                                                      
1    No      10.000000   0.137931  10.000000   0.181728  0.159829
     Yes      0.223776   0.325733  10.000000  10.000000  0.274755
2    No       0.139622   0.162705   0.168859   0.165827  0.164996
     Yes      0.175402   0.148668   0.207893   0.158843  0.166706
3    No       0.187735   0.154661   0.152663   0.084246  0.149671
     Yes     10.000000   0.144995   0.152660   0.204952  0.157543
4    No      10.000000   0.150096   0.148143   0.138919  0.147604
     Yes      0.117750   0.124515   0.193370   0.155410  0.142036
5    No      10.000000  10.000000   0.206928   0.121389  0.178415
     Yes     10.000000   0.106572   0.065660  10.000000  0.086116
6    No      10.000000  10.000000   0.103799   0.173706  0.156229
All           0.169913   0.153152   0.166897   0.161276  0.160803

In [59]:
#A cross-tabulation (or crosstab for short) is a special case of a pivot table that computes group frequencies

In [60]:
from io import StringIO

In [61]:
data = """Sample Nationality Handedness
 .....: 1 USA Right-handed
 .....: 2 Japan Left-handed
 .....: 3 USA Right-handed
 .....: 4 Japan Right-handed
 .....: 5 Japan Left-handed
 .....: 6 Japan Right-handed
 .....: 7 USA Right-handed
 .....: 8 USA Left-handed
 .....: 9 Japan Right-handed
 .....: 10 USA Right-handed"""

In [62]:
data

'Sample Nationality Handedness\n1 USA Right-handed\n2 Japan Left-handed\n3 USA Right-handed\n4 Japan Right-handed\n5 Japan Left-handed\n6 Japan Right-handed\n7 USA Right-handed\n8 USA Left-handed\n9 Japan Right-handed\n10 USA Right-handed'

In [63]:
data = pd.read_table(StringIO(data),sep="\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [64]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [65]:
pd.crosstab(tips["day"], tips["smoker"], margins=True)

smoker,No,Yes,All
day,,,
Fri,4,15,19
Sat,45,42,87
Sun,57,19,76
Thur,45,17,62
All,151,93,244
